In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "ollama simple app"
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Translate this text into {language} you do not have to specify any kind of information in english just translate "),
    ("human", "{input}")
])

In [5]:
parser = StrOutputParser()

In [6]:
chain = prompt | llm | parser

In [7]:
res = chain.invoke({"language":"japanese", "input":"hello my name is anuj"})
print(res)

こんにちは、私の名前はアヌージです。


## simple GenAI app that can remember with sessions

## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [8]:
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store = {} # this stores all the session id

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [9]:
with_message_history = RunnableWithMessageHistory(llm,get_session_history)

In [10]:
config = {"configurable":{"session_id":"chat1"}}

In [11]:
with_message_history.invoke(
    [HumanMessage(content="Hi, I am anuj. i am currently learning gen AI"),
     ],config=config
)

AIMessage(content="Hello Anuj! Nice to meet you! That's great to hear that you're learning about General AI (Gen AI). It's an exciting and rapidly evolving field that has the potential to revolutionize many aspects of our lives.\n\nWhat specific areas of Gen AI are you most interested in learning about? Are you exploring topics like machine learning, natural language processing, computer vision, or something else?\n\nAlso, what's your current level of experience with Gen AI? Are you just starting out, or do you have some background knowledge that you're looking to build upon? I'm here to help and provide guidance as you continue your learning journey!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 129, 'prompt_tokens': 48, 'total_tokens': 177, 'completion_time': 0.282763294, 'prompt_time': 0.00238686, 'queue_time': 0.052695016000000004, 'total_time': 0.285150154}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9a8b91ba77', 'service_t

In [12]:
with_message_history.invoke(
    [HumanMessage(content="what is my name? and do you know what i am learning right now?"),
     ],config=config
)

AIMessage(content='Your name is Anuj, and you are currently learning about General AI (Gen AI).', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 202, 'total_tokens': 221, 'completion_time': 0.05415137, 'prompt_time': 0.010063712, 'queue_time': 0.052049557999999996, 'total_time': 0.064215082}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--25fbc479-87b4-4d4f-8e1c-457480ee4550-0', usage_metadata={'input_tokens': 202, 'output_tokens': 19, 'total_tokens': 221})

In [13]:
config_2 = {"configurable":{"session_id":"chat2"}}
with_message_history.invoke(
    [HumanMessage(content="Hi, do you know my name?"),
     ],config=config_2
)

AIMessage(content="I don't have any information about your name. I'm a large language model, I don't have the ability to recall personal information about individuals, and our conversation just started. I'm happy to chat with you, though! If you'd like to share your name, I'd be happy to use it in our conversation.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 43, 'total_tokens': 110, 'completion_time': 0.146074977, 'prompt_time': 0.002578847, 'queue_time': 0.051620853, 'total_time': 0.148653824}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d80dcf43-6504-4ec5-a281-45170f1566f6-0', usage_metadata={'input_tokens': 43, 'output_tokens': 67, 'total_tokens': 110})

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [39]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability, I want you to answer in this {language}.",),
        MessagesPlaceholder(variable_name="message")
    ]
)
chain = prompt | llm


In [40]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="message")

In [41]:
config3 = {"configurable":{"session_id":"chat_6"}}

In [42]:
res = with_message_history.invoke({
    "message":[HumanMessage(content="Hi i am anuj.")],"language":"japanese"
},
config=config3
)
res.content

'(Konnichiwa Anuj-san!)\n\n\n(Anata wa Anuj desu ne?)'

In [43]:
res = with_message_history.invoke({
    "message":[HumanMessage(content="what is my name")],"language":"japanese"
},
config=config3
)
res.content

'(Anata no namae wa Anuj desu.)'

## Managing the conversation history